# Setup

In [1]:
import re
import numexpr
import time
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, FewShotPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.example_selectors import LengthBasedExampleSelector
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from IPython.display import display, Markdown

In [2]:
def print_text(text):
    return display(Markdown(f'<div style="font-size: 17px;">\n\n{text}\n\n</div>'))

# LLM Setup

In [3]:
# Use HuggingFaceEndpoint - automatic routing based on availability
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    task="text-generation",
    max_new_tokens=300,
    temperature=0.7,
)

chat_llm = ChatHuggingFace(llm=llm)

# Langchain Quick Intro

## Asking a simple question

In [4]:
# Build prompt template
template = """Question: {question}

Answer: """

prompt = PromptTemplate(template=template, input_variables=["question"])

# we chain together the prompt -> LLM with LCEL (more on this later)
llm_chain = prompt | chat_llm

question = "Which NFL team won the Super Bowl in the 2010 season?"

text = (llm_chain.invoke(question))
time.sleep(3)
print_text(text.content)

<div style="font-size: 17px;">

The New Orleans Saints won Super Bowl XLIV (44) in the 2010 season. They defeated the Indianapolis Colts with a score of 31-17.

</div>

## Asking multiple questions

In [5]:
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]

res = llm_chain.batch(qs)
time.sleep(3)

In [6]:
for question, response in zip(qs, res):
    print_text("="*100)
    print_text(f"QUESTION: {question['question']}")
    print_text(f"RESPONSE: {response.content}")
    print_text("="*100 + "\n")

<div style="font-size: 17px;">

====================================================================================================

</div>

<div style="font-size: 17px;">

QUESTION: Which NFL team won the Super Bowl in the 2010 season?

</div>

<div style="font-size: 17px;">

RESPONSE: The New Orleans Saints won the Super Bowl in the 2009 season, not the 2010 season. However, the Green Bay Packers won the Super Bowl in the 2010 season.

</div>

<div style="font-size: 17px;">

====================================================================================================


</div>

<div style="font-size: 17px;">

====================================================================================================

</div>

...

<div style="font-size: 17px;">

====================================================================================================


</div>

<div style="font-size: 17px;">

====================================================================================================

</div>

<div style="font-size: 17px;">

QUESTION: How many eyes does a blade of grass have?

</div>

<div style="font-size: 17px;">

RESPONSE: The correct answer is 1. Each blade of grass has a single tip that functions as an eye, known as an apical meristem. This eye is responsible for growth and photosynthesis. However, if we consider the entire root system, it can have multiple "eyes" or growth points.

</div>

<div style="font-size: 17px;">

====================================================================================================


</div>

# Langchain Prompt Templates

## Simple prompt

In [7]:
prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: """

In [8]:
text = chat_llm.invoke(prompt)
time.sleep(3)
print_text(text.content)

<div style="font-size: 17px;">

The `transformers` library from Hugging Face, the `openai` library from OpenAI, and the `cohere` library from Cohere offer Large Language Models (LLMs).

</div>

## Prompt Templates

In [9]:
template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

In [10]:
final_prompt = prompt_template.format(
        query="Which libraries and model providers offer LLMs?"
    )
print_text(final_prompt)

<div style="font-size: 17px;">

Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: 

</div>

In [11]:
output_parser = StrOutputParser()
chain = prompt_template | chat_llm | output_parser

response = chain.invoke({"query": "Which libraries and model providers offer LLMs?"})
time.sleep(3)
print_text(response)

<div style="font-size: 17px;">

The `transformers` library from Hugging Face, the `openai` library, and the `cohere` library offer Large Language Models (LLMs).

</div>

## Few-shot Training

In [12]:
prompt = PromptTemplate(
    template=(
        "Create an FAQ in Markdown based on the following questions and answers:\n"
        "Q1: What is your return policy?\n"
        "A1: We accept returns within 30 days with receipt.\n"
        "Q2: Do you ship internationally?\n"
        "A2: Yes, we ship to over 50 countries.\n"
        "Q3: How can I track my order?\n"
        "A3: Use the tracking link in your confirmation email."
    ),
    input_variables=[]  # No variables needed yet
)

chain = prompt | chat_llm | output_parser
response = chain.invoke({})  # Empty dict since no input variables
time.sleep(3)
print_text(response)

<div style="font-size: 17px;">

FAQ
================

### Returning an Order

#### Q: What is your return policy?

A: We accept returns within 30 days with a valid receipt. Please contact us for further instructions on the return process.

### Shipping and Delivery

#### Q: Do you ship internationally?

A: Yes, we ship to over 50 countries worldwide. Please note that international shipping rates and delivery times may vary.

### Tracking Your Order

#### Q: How can I track my order?

A: To track your order, simply use the tracking link provided in your confirmation email. This will allow you to monitor the status of your shipment in real-time.

</div>

In [13]:
prompt_str = """
Create a structured Markdown FAQ with anchor links, headers, and formatting conventions for readability.

**Example:**

Example input:
Q1: What is your return policy?
A1: We accept returns within 30 days with the original receipt.
Q2: Do you ship internationally?
A2: Yes, we ship to over 50 countries worldwide.
Q3: How can I track my order?
A3: After your order is shipped, you'll receive a tracking link via email.

Example Output:

# Frequently Asked Questions

## [1. What is your return policy?](#1-what-is-your-return-policy)

We accept returns within **30 days** with the original **receipt**.

## [2. Do you ship internationally?](#2-do-you-ship-internationally)

Yes, we ship to over **50 countries** worldwide.

## [3. How can I track my order?](#3-how-can-i-track-my-order)

After your order is shipped, you'll receive a **tracking link** via email.

---

Now generate the FAQ section for this data:
Q1: What payment methods do you accept?
A1: We accept Visa, Mastercard, PayPal, and Apple Pay.
Q2: Can I change my shipping address after ordering?
A2: Only if your order hasn't shipped yet. Contact support ASAP.
Q3: Do you offer gift wrapping?
A3: Yes! You can select gift wrapping during checkout.

"""

# Create a PromptTemplate with no input variables (static prompt)
prompt = PromptTemplate(template=prompt_str, input_variables=[])

chain = prompt | chat_llm | output_parser
response = chain.invoke({})  # Empty dict since no input variables
time.sleep(3)
print_text(response)

<div style="font-size: 17px;">

# Frequently Asked Questions

## [1. What payment methods do you accept?](#1-what-payment-methods-do-you-accept)

We accept **Visa**, **Mastercard**, **PayPal**, and **Apple Pay**.

## [2. Can I change my shipping address after ordering?](#2-can-i-change-my-shipping-address-after-ordering)

Only if your order hasn't shipped yet. Contact support **ASAP**.

## [3. Do you offer gift wrapping?](#3-do-you-offer-gift-wrapping)

Yes! You can select gift wrapping during **checkout**.

---

You can easily navigate to each section using the anchor links provided. Just click on the section title to scroll to that part of the FAQ.

</div>

## Few Shot Prompt Templates

In [14]:
# Simplified examples with just 2 variables instead of 9
examples = [
    {
        "input": (
            "Q1: What is your return policy?\n"
            "A1: We accept returns within 30 days with the original receipt."
        ),
        "output": (
            "## [1. What is your return policy?](#1-what-is-your-return-policy)\n\n"
            "We accept returns within **30 days** with the original **receipt**."
        )
    },
    {
        "input": (
            "Q2: Do you ship internationally?\n"
            "A2: Yes, we ship to over 50 countries worldwide."
        ),
        "output": (
            "## [2. Do you ship internationally?](#2-do-you-ship-internationally)\n\n"
            "Yes, we ship to over **50 countries** worldwide."
        )
    },
    {
        "input": (
            "Q3: How can I track my order?\n"
            "A3: After your order is shipped, you'll receive a tracking link via email."
        ),
        "output": (
            "## [3. How can I track my order?](#3-how-can-i-track-my-order)\n\n"
            "After your order is shipped, you'll receive a **tracking link** via email."
        )
    }
]

# Much simpler example template with only 2 variables
example_template = """Example input:
{input}

Example Output:
# Frequently Asked Questions\n\n
{output}
"""

# Create prompt template for examples
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template=example_template
)

# Instructions prefix
prefix = """Create a structured Markdown FAQ with anchor links, headers, and formatting conventions for readability.
Make sure to bold key terms and important information in the answers.

**Example:**
"""

# User input format and instructions
suffix = """
Now generate the FAQ section for this data:
{input}
"""

# Create the few-shot prompt template
faq_few_shot = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input"],
    example_separator="\n"
)

In [15]:
# See what the prompt looks like before sending to LLM
user_input = """Q1: What payment methods do you accept?
A1: We accept Visa, Mastercard, PayPal, and Apple Pay.
Q2: Can I change my shipping address after ordering?
A2: Only if your order hasn't shipped yet. Contact support ASAP.
Q3: Do you offer gift wrapping?
A3: Yes! You can select gift wrapping during checkout."""

formatted_prompt = faq_few_shot.format(input=user_input)
print_text(formatted_prompt)

<div style="font-size: 17px;">

Create a structured Markdown FAQ with anchor links, headers, and formatting conventions for readability.
Make sure to bold key terms and important information in the answers.

**Example:**

Example input:
Q1: What is your return policy?
A1: We accept returns within 30 days with the original receipt.

Example Output:
# Frequently Asked Questions


## [1. What is your return policy?](#1-what-is-your-return-policy)

We accept returns within **30 days** with the original **receipt**.

Example input:
Q2: Do you ship internationally?
A2: Yes, we ship to over 50 countries worldwide.

Example Output:
# Frequently Asked Questions


## [2. Do you ship internationally?](#2-do-you-ship-internationally)

Yes, we ship to over **50 countries** worldwide.

Example input:
Q3: How can I track my order?
A3: After your order is shipped, you'll receive a tracking link via email.

Example Output:
# Frequently Asked Questions


## [3. How can I track my order?](#3-how-can-i-track-my-order)

After your order is shipped, you'll receive a **tracking link** via email.


Now generate the FAQ section for this data:
Q1: What payment methods do you accept?
A1: We accept Visa, Mastercard, PayPal, and Apple Pay.
Q2: Can I change my shipping address after ordering?
A2: Only if your order hasn't shipped yet. Contact support ASAP.
Q3: Do you offer gift wrapping?
A3: Yes! You can select gift wrapping during checkout.


</div>

In [16]:
chain = faq_few_shot | chat_llm | output_parser

# Execute with LCEL chain
response = chain.invoke({"input": user_input})
time.sleep(3)

# Display the formatted FAQ (response is already a clean string)
print_text(response)

<div style="font-size: 17px;">

# Frequently Asked Questions

## [1. What payment methods do you accept?](#1-what-payment-methods-do-you-accept)

We accept **Visa**, **Mastercard**, **PayPal**, and **Apple Pay**.

## [2. Can I change my shipping address after ordering?](#2-can-i-change-my-shipping-address-after-ordering)

**Only if your order hasn't shipped yet**. Contact our support team as soon as possible to make any necessary changes.

## [3. Do you offer gift wrapping?](#3-do-you-offer-gift-wrapping)

Yes! You can select gift wrapping during **checkout**.

</div>

## Dynamic inclusion/exclusion of examples in FewShotPromptTemplate using LengthBasedExampleSelector

In [17]:
# note we're adding more FAQ examples to each individual example here
# (how much you add isn't necessarily important - just that you're aligning
# examples as closely as possible to your use-case)
examples = [
    {
        "input": """Q1: What is your return policy?
A1: We accept returns within 30 days with the original receipt.
Q2: Do you ship internationally?
A2: Yes, we ship to over 50 countries worldwide.
Q3: How can I track my order?
A3: After your order is shipped, you'll receive a tracking link via email.""",
        
        "output": """## [1. What is your return policy?](#1-what-is-your-return-policy)

We accept returns within **30 days** with the original **receipt**.

## [2. Do you ship internationally?](#2-do-you-ship-internationally)

Yes, we ship to over **50 countries** worldwide.

## [3. How can I track my order?](#3-how-can-i-track-my-order)

After your order is shipped, you'll receive a **tracking link** via email.

---"""
    },
    {
        "input": """Q1: What payment methods do you accept?
A1: We accept Visa, Mastercard, PayPal, and Apple Pay.
Q2: Can I change my shipping address after ordering?
A2: Only if your order hasn't shipped yet. Contact support ASAP.
Q3: Do you offer gift wrapping?
A3: Yes! You can select gift wrapping during checkout.""",
        
        "output": """## [1. What payment methods do you accept?](#1-what-payment-methods-do-you-accept)

We accept **Visa**, **Mastercard**, **PayPal**, and **Apple Pay**.

## [2. Can I change my shipping address after ordering?](#2-can-i-change-my-shipping-address-after-ordering)

Only if your order hasn't shipped yet. Contact support **ASAP**.

## [3. Do you offer gift wrapping?](#3-do-you-offer-gift-wrapping)

Yes! You can select **gift wrapping** during checkout.

---"""
    },
    {
        "input": """Q1: What are your store hours?
A1: We're open Monday through Friday from 9am to 9pm, and weekends from 10am to 7pm.
Q2: Do you offer price matching?
A2: Yes, we'll match any price from authorized retailers for identical products.
Q3: What is your warranty policy?
A3: All electronics come with a standard 1-year manufacturer warranty.""",
        
        "output": """## [1. What are your store hours?](#1-what-are-your-store-hours)

We're open **Monday through Friday** from **9am to 9pm**, and **weekends** from **10am to 7pm**.

## [2. Do you offer price matching?](#2-do-you-offer-price-matching)

Yes, we'll match any price from **authorized retailers** for **identical products**.

## [3. What is your warranty policy?](#3-what-is-your-warranty-policy)

All electronics come with a standard **1-year manufacturer warranty**.

---"""
    },
    {
        "input": """Q1: How do I contact customer service?
A1: You can reach us at support@example.com or call 555-123-4567.
Q2: Do you offer same-day delivery?
A2: Yes, for orders placed before 2pm in selected metro areas.
Q3: How do I cancel an order?
A3: Log into your account and cancel within 1 hour of placing the order.""",
        
        "output": """## [1. How do I contact customer service?](#1-how-do-i-contact-customer-service)

You can reach us at **support@example.com** or call **555-123-4567**.

## [2. Do you offer same-day delivery?](#2-do-you-offer-same-day-delivery)

Yes, for orders placed before **2pm** in selected **metro areas**.

## [3. How do I cancel an order?](#3-how-do-i-cancel-an-order)

Log into your account and cancel within **1 hour** of placing the order.

---"""
    },
    {
        "input": """Q1: Do you offer student discounts?
A1: Yes, students with valid ID receive 15% off all purchases.
Q2: What is your privacy policy?
A2: We never share your personal information with third parties without consent.
Q3: Are your products environmentally friendly?
A3: We use recyclable packaging and offer carbon-neutral shipping options.""",
        
        "output": """## [1. Do you offer student discounts?](#1-do-you-offer-student-discounts)

Yes, students with valid ID receive **15% off** all purchases.

## [2. What is your privacy policy?](#2-what-is-your-privacy-policy)

We **never share** your personal information with third parties without consent.

## [3. Are your products environmentally friendly?](#3-are-your-products-environmentally-friendly)

We use **recyclable packaging** and offer **carbon-neutral shipping** options.

---"""
    },
    {
        "input": """Q1: How do I apply for a refund?
A1: Submit your request through our customer portal with your order number and reason.
Q2: Do you have a loyalty program?
A2: Yes! Earn 1 point for every dollar spent and redeem for discounts.
Q3: What are the system requirements?
A3: Our software requires Windows 10/11 or macOS 10.15+, 8GB RAM, and 2GB storage.""",
        
        "output": """## [1. How do I apply for a refund?](#1-how-do-i-apply-for-a-refund)

Submit your request through our **customer portal** with your **order number** and reason.

## [2. Do you have a loyalty program?](#2-do-you-have-a-loyalty-program)

Yes! Earn **1 point** for every dollar spent and redeem for **discounts**.

## [3. What are the system requirements?](#3-what-are-the-system-requirements)

Our software requires **Windows 10/11** or **macOS 10.15+**, **8GB RAM**, and **2GB storage**.

---"""
    },
    {
        "input": """Q1: How long does shipping take?
A1: Standard shipping takes 3-5 business days, and express shipping takes 1-2 business days.
Q2: Do you have physical stores?
A2: Yes, we have 12 locations across North America. Find the nearest one on our website.
Q3: How do I reset my password?
A3: Click 'Forgot Password' on the login page and follow the email instructions.""",

        "output": """## [1. How long does shipping take?](#1-how-long-does-shipping-take)

Standard shipping takes **3-5 business days**, and express shipping takes **1-2 business days**.

## [2. Do you have physical stores?](#2-do-you-have-physical-stores)

Yes, we have **12 locations** across North America. Find the nearest one on our **website**.

## [3. How do I reset my password?](#3-how-do-i-reset-my-password)

Click **'Forgot Password'** on the login page and follow the email instructions.

---"""
    },
    {
        "input": """Q1: What's your wholesale policy?
A1: For orders over $500, contact our wholesale department for special pricing.
Q2: How can I become a vendor?
A2: Fill out the vendor application form on our Partners page for consideration.
Q3: Do you offer installation services?
A3: Yes, professional installation is available for an additional fee in most areas.""",
        
        "output": """## [1. What's your wholesale policy?](#1-whats-your-wholesale-policy)

For orders over **$500**, contact our **wholesale department** for special pricing.

## [2. How can I become a vendor?](#2-how-can-i-become-a-vendor)

Fill out the **vendor application form** on our **Partners page** for consideration.

## [3. Do you offer installation services?](#3-do-you-offer-installation-services)

Yes, **professional installation** is available for an additional fee in **most areas**.

---"""
    }
]

In [18]:
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=700  # this sets the max length that examples should be
)

In [19]:
some_text = "There are a total of 8 words here.\nPlus 6 here, totaling 14 words."

words = re.split('[\n ]', some_text)
print(words, len(words))

['There', 'are', 'a', 'total', 'of', '8', 'words', 'here.', 'Plus', '6', 'here,', 'totaling', '14', 'words.'] 14


In [20]:
# Then create the dynamic prompt template
dynamic_faq_prompt = FewShotPromptTemplate(
    example_selector=example_selector,  # use example_selector instead of examples
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input"],  # simplified to just one variable
    example_separator="\n"
)

In [21]:
# Using simplified input format
user_input = """Q1: What payment methods do you accept?
A1: We accept Visa, Mastercard, PayPal, and Apple Pay.
Q2: Can I change my shipping address after ordering?
A2: Only if your order hasn't shipped yet. Contact support ASAP.
Q3: Do you offer gift wrapping?
A3: Yes! You can select gift wrapping during checkout."""

formatted_prompt = dynamic_faq_prompt.format(input=user_input)
print_text(formatted_prompt)

<div style="font-size: 17px;">

Create a structured Markdown FAQ with anchor links, headers, and formatting conventions for readability.
Make sure to bold key terms and important information in the answers.

**Example:**

Example input:
Q1: What is your return policy?
A1: We accept returns within 30 days with the original receipt.
Q2: Do you ship internationally?
A2: Yes, we ship to over 50 countries worldwide.
Q3: How can I track my order?
A3: After your order is shipped, you'll receive a tracking link via email.

Example Output:
# Frequently Asked Questions


## [1. What is your return policy?](#1-what-is-your-return-policy)

We accept returns within **30 days** with the original **receipt**.

## [2. Do you ship internationally?](#2-do-you-ship-internationally)

Yes, we ship to over **50 countries** worldwide.

## [3. How can I track my order?](#3-how-can-i-track-my-order)

After your order is shipped, you'll receive a **tracking link** via email.

---

Example input:
Q1: What payment methods do you accept?
A1: We accept Visa, Mastercard, PayPal, and Apple Pay.
Q2: Can I change my shipping address after ordering?
A2: Only if your order hasn't shipped yet. Contact support ASAP.
Q3: Do you offer gift wrapping?
A3: Yes! You can select gift wrapping during checkout.

Example Output:
# Frequently Asked Questions


## [1. What payment methods do you accept?](#1-what-payment-methods-do-you-accept)

We accept **Visa**, **Mastercard**, **PayPal**, and **Apple Pay**.

## [2. Can I change my shipping address after ordering?](#2-can-i-change-my-shipping-address-after-ordering)

Only if your order hasn't shipped yet. Contact support **ASAP**.

## [3. Do you offer gift wrapping?](#3-do-you-offer-gift-wrapping)

Yes! You can select **gift wrapping** during checkout.

---

Example input:
Q1: What are your store hours?
A1: We're open Monday through Friday from 9am to 9pm, and weekends from 10am to 7pm.
Q2: Do you offer price matching?
A2: Yes, we'll match any price from authorized retailers for identical products.
Q3: What is your warranty policy?
A3: All electronics come with a standard 1-year manufacturer warranty.

Example Output:
# Frequently Asked Questions


## [1. What are your store hours?](#1-what-are-your-store-hours)

We're open **Monday through Friday** from **9am to 9pm**, and **weekends** from **10am to 7pm**.

## [2. Do you offer price matching?](#2-do-you-offer-price-matching)

Yes, we'll match any price from **authorized retailers** for **identical products**.

## [3. What is your warranty policy?](#3-what-is-your-warranty-policy)

All electronics come with a standard **1-year manufacturer warranty**.

---

Example input:
Q1: How do I contact customer service?
A1: You can reach us at support@example.com or call 555-123-4567.
Q2: Do you offer same-day delivery?
A2: Yes, for orders placed before 2pm in selected metro areas.
Q3: How do I cancel an order?
A3: Log into your account and cancel within 1 hour of placing the order.

Example Output:
# Frequently Asked Questions


## [1. How do I contact customer service?](#1-how-do-i-contact-customer-service)

You can reach us at **support@example.com** or call **555-123-4567**.

## [2. Do you offer same-day delivery?](#2-do-you-offer-same-day-delivery)

Yes, for orders placed before **2pm** in selected **metro areas**.

## [3. How do I cancel an order?](#3-how-do-i-cancel-an-order)

Log into your account and cancel within **1 hour** of placing the order.

---

Example input:
Q1: Do you offer student discounts?
A1: Yes, students with valid ID receive 15% off all purchases.
Q2: What is your privacy policy?
A2: We never share your personal information with third parties without consent.
Q3: Are your products environmentally friendly?
A3: We use recyclable packaging and offer carbon-neutral shipping options.

Example Output:
# Frequently Asked Questions


## [1. Do you offer student discounts?](#1-do-you-offer-student-discounts)

Yes, students with valid ID receive **15% off** all purchases.

## [2. What is your privacy policy?](#2-what-is-your-privacy-policy)

We **never share** your personal information with third parties without consent.

## [3. Are your products environmentally friendly?](#3-are-your-products-environmentally-friendly)

We use **recyclable packaging** and offer **carbon-neutral shipping** options.

---


Now generate the FAQ section for this data:
Q1: What payment methods do you accept?
A1: We accept Visa, Mastercard, PayPal, and Apple Pay.
Q2: Can I change my shipping address after ordering?
A2: Only if your order hasn't shipped yet. Contact support ASAP.
Q3: Do you offer gift wrapping?
A3: Yes! You can select gift wrapping during checkout.


</div>

In [22]:
dynamic_chain = dynamic_faq_prompt | chat_llm | output_parser

# Execute with LCEL chain
response = dynamic_chain.invoke({"input": user_input})
time.sleep(3)
print_text(response)

<div style="font-size: 17px;">

# Frequently Asked Questions


## [1. What payment methods do you accept?](#1-what-payment-methods-do-you-accept)

We accept **Visa**, **Mastercard**, **PayPal**, and **Apple Pay**.

## [2. Can I change my shipping address after ordering?](#2-can-i-change-my-shipping-address-after-ordering)

Only if your order hasn't shipped yet. **Contact support ASAP**.

## [3. Do you offer gift wrapping?](#3-do-you-offer-gift-wrapping)

Yes! You can select **gift wrapping** during checkout.

</div>

## LengthBasedExampleSelector working when user input is much longer and using more of the context window

In [23]:
# Using simplified input format with very long questions and answers
user_input = """Q1: What are all the different payment methods that you accept for online purchases, including credit cards, digital wallets, bank transfers, and any special financing options that might be available for customers?
A1: We accept a comprehensive range of payment methods to accommodate all our customers' preferences and needs. For credit cards, we accept Visa, Mastercard, American Express, and Discover. We also support digital wallet payments through PayPal, Apple Pay, Google Pay, Samsung Pay, and Amazon Pay. Additionally, we offer bank transfer options including ACH transfers, wire transfers, and direct debit for customers who prefer traditional banking methods. For larger purchases, we provide financing options through Affirm, Klarna, and our own in-house financing program with flexible payment plans ranging from 6 to 36 months. We also accept cryptocurrency payments including Bitcoin, Ethereum, and several other major cryptocurrencies for tech-savvy customers.
Q2: Is it possible for me to modify or completely change my shipping address after I have already placed and confirmed my order, and if so, what are the specific conditions, timeframes, and procedures that I need to follow?
A2: Yes, it is possible to modify your shipping address, but this depends entirely on the current status of your order in our fulfillment process. If your order has not yet been processed by our warehouse team and is still in 'pending' or 'confirmed' status, you can easily change the shipping address by logging into your account and accessing the order management section. However, once your order enters the 'processing' phase and our warehouse team begins preparing your items for shipment, address changes become much more complicated and may not be possible. If your order has already shipped, unfortunately we cannot redirect the package to a different address, but you can contact the shipping carrier directly to arrange for package interception or redirection services, though additional fees may apply. For the best chance of successful address modification, we strongly recommend contacting our customer support team immediately at support@company.com or calling our toll-free number.
Q3: Do you provide gift wrapping services for the items that I purchase, and if you do, what are the different options available, what are the costs involved, and can I include personalized messages or special requests?
A3: Absolutely! We offer comprehensive gift wrapping services to make your purchases extra special for any occasion. We have several gift wrapping options available: our standard gift wrap features elegant wrapping paper in various colors and patterns with matching ribbon and a bow for an additional $4.99 per item. Our premium gift wrap option includes luxury wrapping paper, silk ribbon, and decorative embellishments for $9.99 per item. For special occasions, we offer themed wrapping for holidays, birthdays, weddings, and baby showers at $7.99 per item. You can also add personalized gift messages up to 250 characters at no additional cost, and we'll include them on beautiful greeting cards. For an extra $2.99, you can upload custom messages or even photos to be printed on special cards. All gift-wrapped items are carefully packaged to ensure they arrive in perfect condition, and we offer discrete packaging options if you're sending gifts directly to recipients."""

In [24]:
formatted_prompt = dynamic_faq_prompt.format(input=user_input)
print(formatted_prompt)

Create a structured Markdown FAQ with anchor links, headers, and formatting conventions for readability.
Make sure to bold key terms and important information in the answers.

**Example:**

Example input:
Q1: What is your return policy?
A1: We accept returns within 30 days with the original receipt.
Q2: Do you ship internationally?
A2: Yes, we ship to over 50 countries worldwide.
Q3: How can I track my order?
A3: After your order is shipped, you'll receive a tracking link via email.

Example Output:
# Frequently Asked Questions


## [1. What is your return policy?](#1-what-is-your-return-policy)

We accept returns within **30 days** with the original **receipt**.

## [2. Do you ship internationally?](#2-do-you-ship-internationally)

Yes, we ship to over **50 countries** worldwide.

## [3. How can I track my order?](#3-how-can-i-track-my-order)

After your order is shipped, you'll receive a **tracking link** via email.

---


Now generate the FAQ section for this data:
Q1: What are all 

In [25]:
# Execute with LCEL chain
response = dynamic_chain.invoke(user_input)
time.sleep(3)

# Display the formatted FAQ (response is already a clean string)
print_text(response)

<div style="font-size: 17px;">

# Frequently Asked Questions

## [1. What payment methods do you accept for online purchases?](#1-what-payment-methods-do-you-accept-for-online-purchases)

We accept a comprehensive range of payment methods to accommodate all our customers' preferences and needs. For **credit cards**, we accept **Visa**, **Mastercard**, **American Express**, and **Discover**. We also support digital wallet payments through **PayPal**, **Apple Pay**, **Google Pay**, **Samsung Pay**, and **Amazon Pay**. Additionally, we offer **bank transfer** options including **ACH transfers**, **wire transfers**, and **direct debit** for customers who prefer traditional banking methods. For larger purchases, we provide financing options through **Affirm**, **Klarna**, and our own in-house financing program with flexible payment plans ranging from **6 to 36 months**. We also accept **cryptocurrency payments** including **Bitcoin**, **Ethereum**, and several other major cryptocurrencies for tech-savvy customers.

## [2. Can I modify or change my shipping address after I have placed and confirmed my order?](#2-can-i-modify-or-change-my-shipping-address-after-i-have-placed-and-confirmed-my-order)

Yes, it is possible to modify your shipping address, but this depends entirely on the current status of your order in our fulfillment process. If your order has not yet been processed by our warehouse team and is still in '**pending**' or '**

</div>

# Langchain New Prompt Templates

## New Prompt Templates

In [26]:
# Create the message templates
system_template = "You are a helpful assistant."
human_template = "{input}"

# Create the prompt template
prompt = ChatPromptTemplate([
    ("system", system_template),
    ("human", human_template)
])

# Create the chain
chain = prompt | chat_llm | StrOutputParser()

# Test the chain
result = chain.invoke({"input": "Hi AI, how are you today?"})
time.sleep(3)
print_text(result)

<div style="font-size: 17px;">

I'm doing well, thank you for asking. I'm a large language model, so I don't have feelings like humans do, but I'm always ready to help answer any questions or provide information on a wide range of topics. How can I assist you today?

</div>

In [27]:
# Create initial messages
prompt = ChatPromptTemplate([
    ("system", "You are a helpful assistant."),
    ("human", "Hi AI, how are you today?"),
    ("ai", "I'm great thank you. How can I help you?"),
    ("human", "I'd like to understand string theory.")
])

# Create the chain using LCEL pipe syntax
chain = prompt | chat_llm | StrOutputParser()

# Get response using LCEL
res = chain.invoke({})
time.sleep(3)
print_text(res)

<div style="font-size: 17px;">

String theory is a fundamental concept in modern physics that attempts to unify the principles of quantum mechanics and general relativity. It's a complex and mind-bending topic, but I'll try to break it down for you in simple terms.

**What is String Theory?**

String theory proposes that the fundamental building blocks of the universe are not particles, but tiny, vibrating strings. These strings are thought to be the basic units of matter and energy, and they can vibrate at different frequencies to create various particles, like electrons and photons.

**Key Concepts:**

1. **Strings:** The fundamental objects in string theory are one-dimensional strings, which are too small to be observed directly. They're like tiny, vibrating filaments that exist at the quantum level.
2. **Vibrational Modes:** The strings vibrate at different frequencies, analogous to the strings on a violin. Each frequency corresponds to a specific particle, like an electron or a photon.
3. **Extra Dimensions:** String theory requires the existence of more than the three spatial dimensions (length, width, and height) and one time dimension we experience in everyday life. These additional dimensions are "curled up" or "compactified" so tightly that they're not directly observable.
4. **Superstrings:** There are two main types of strings: open strings (with free ends) and closed strings (forming a loop). Open strings can vibrate in different modes to create particles, while closed strings can be thought of as a

</div>

In [28]:
# For the follow-up question, we can extend the existing prompt
prompt.extend([
    ("ai", res),  # Previous AI response
    ("human", "Why do physicists believe it can produce a 'unified theory'?")
])

# Create the chain using LCEL pipe syntax
chain = prompt | chat_llm | StrOutputParser()

# Get response
result = chain.invoke({})
time.sleep(3)
print_text(result)

<div style="font-size: 17px;">

Physicists believe that string theory can produce a unified theory for several reasons:

1. **Unifying Quantum Mechanics and General Relativity:** String theory attempts to merge two major areas of physics: quantum mechanics (describing the behavior of particles at the atomic and subatomic level) and general relativity (describing gravity and the large-scale structure of the universe). General relativity is a theory of gravity, while quantum mechanics is a theory of particles. String theory tries to reconcile these two theories by showing how gravity and particles are manifestations of the same underlying phenomenon.

2. **Including All Fundamental Forces:** String theory attempts to unify the four fundamental forces of nature:
	* Gravity (described by general relativity)
	* Electromagnetism
	* The strong nuclear force (which holds quarks together inside protons and neutrons)
	* The weak nuclear force (responsible for certain types of radioactive decay)

3. **Predicting New Particles:** String theory predicts the existence of new particles that are not yet observed in experiments. These particles could be discovered in future experiments, which would provide evidence for string theory.

4. **Solving the Black Hole Information Paradox:** String theory provides an explanation for the black hole information paradox, which questions what happens to the information contained in matter that falls into a black hole. String theory suggests that this information is preserved, but in a form that's not yet understood.

5. **Mathematical Consistency:** String theory has

</div>

In [29]:
languages = [
    "English",
    "Esperanto",
    "Spanish",
    # "French",
    # "German",
    # "Italian",
    # "Portuguese",
    # "Dutch",
    # "Russian",
    # "Chinese (Simplified)",
    # "Chinese (Traditional)",
    # "Japanese",
    # "Korean",
    # "Arabic",
    # "Hindi",
    # "Turkish",
    # "Swedish",
    # "Danish",
    # "Norwegian",
    # "Finnish",
    # "Polish",
    # "Czech",
    # "Hungarian",
    # "Greek",
    # "Hebrew",
    # "Vietnamese",
    # "Thai"
]

In [30]:
from langchain_core.prompts.chat import HumanMessagePromptTemplate, SystemMessagePromptTemplate

# Create the prompt template
human_template = HumanMessagePromptTemplate.from_template(
    "Translate this input <INPUT_START> {input} <INPUT_END>  into {language}. Do not include any other text in your response."
)
chat_prompt = ChatPromptTemplate([human_template])

# Format with dynamic input
chat_prompt_value = chat_prompt.format_prompt(
    input="I hope when you come the weather will be clement.", # Extra points if you get the reference.
    language="Spanish"
)

chat_prompt_value

ChatPromptValue(messages=[HumanMessage(content='Translate this input <INPUT_START> I hope when you come the weather will be clement. <INPUT_END>  into Spanish. Do not include any other text in your response.', additional_kwargs={}, response_metadata={})])

In [31]:
chat_prompt_value.to_messages()

[HumanMessage(content='Translate this input <INPUT_START> I hope when you come the weather will be clement. <INPUT_END>  into Spanish. Do not include any other text in your response.', additional_kwargs={}, response_metadata={})]

In [32]:
chat_prompt_value.to_string()

'Human: Translate this input <INPUT_START> I hope when you come the weather will be clement. <INPUT_END>  into Spanish. Do not include any other text in your response.'

In [33]:
human_template = HumanMessagePromptTemplate.from_template(
    "Translate this input '{input}' into {language}. Do not include any other text in your response."
)
system_template = SystemMessagePromptTemplate.from_template("You are a helpful assistant.")

# Create the chain using LCEL pipe syntax
chain = (
    ChatPromptTemplate([system_template, human_template])
    | chat_llm
    | StrOutputParser()
)

# Loop through each language
for language in languages:
    print_text(f"\n=== Response in {language} ===")

    # Invoke the chain with our inputs
    result = chain.invoke({
        "input": "I hope when you come the weather will be clement.",
        "language": language
    })
    time.sleep(3)

    print_text(result)
    print_text("=" * 50)  # Separator for readability

<div style="font-size: 17px;">


=== Response in English ===

</div>

<div style="font-size: 17px;">

I hope when you come the weather will be nice.

</div>

<div style="font-size: 17px;">

==================================================

</div>

<div style="font-size: 17px;">


=== Response in Esperanto ===

</div>

<div style="font-size: 17px;">

Mi esperas, ke kiam vi venos, la vetero estos milda.

</div>

<div style="font-size: 17px;">

==================================================

</div>

<div style="font-size: 17px;">


=== Response in Spanish ===

</div>

<div style="font-size: 17px;">

Espero que cuando vengas el tiempo será agradable.

</div>

<div style="font-size: 17px;">

==================================================

</div>

In [34]:
# Create few-shot examples for technical content formatting
system_template = SystemMessagePromptTemplate.from_template(
    """You are a technical translator. You must maintain the exact same format and structure in your translations.
    Only translate the explanatory text, keeping all technical terms, numbers, and formatting unchanged.

    Example input and output pairs:

    Input: "Error 404: Page not found"
    Output: "Error 404: Página no encontrada"

    Input: "Status: 200 OK
    Response: {{
        'data': 'success',
        'message': 'Operation completed'
    }}"
    Output: "Status: 200 OK
    Response: {{
        'data': 'success',
        'message': 'Operación completada'
    }}"
    """
)

# Example of a technical input
human_template = HumanMessagePromptTemplate.from_template(
    """Translate this technical information to {language}:

    Status: 500 Internal Server Error
    Response: {{
        'error': 'Database connection failed',
        'code': 'DB_001',
        'timestamp': '2024-03-20T10:30:00Z'
    }}

    Technical Note: This error occurs when the application cannot connect to the database.
    """
)

# Create the chain using LCEL pipe syntax
chain = (
    ChatPromptTemplate([system_template, human_template])
    | chat_llm
    | StrOutputParser()
)

# Loop through each language
for language in languages:
    print_text(f"\n=== Technical Translation in {language} ===")

    # Invoke the chain with our input
    result = chain.invoke({"language": language})
    time.sleep(3)

    print_text(result)
    print_text("=" * 80)  # Separator for readability

<div style="font-size: 17px;">


=== Technical Translation in English ===

</div>

<div style="font-size: 17px;">

    Status: 500 Internal Server Error
    Response: {
        'error': 'Database connection failed',
        'code': 'DB_001',
        'timestamp': '2024-03-20T10:30:00Z'
    }

    Technical Note: This error occurs when the application fails to connect to the database.

</div>

<div style="font-size: 17px;">

================================================================================

</div>

<div style="font-size: 17px;">


=== Technical Translation in Esperanto ===

</div>

<div style="font-size: 17px;">

Status: 500 Interna Servoba Krizo
    Respondo: {
        'eraro': 'Databazo konektiĝo malsukcesis',
        'kodo': 'DB_001',
        'tempo': '2024-03-20T10:30:00Z'
    }

    Teknika Noto: Ĉi tiu eraro okazas, kiam la aplikaĵo ne povas konektiĝ al la databazo.

</div>

<div style="font-size: 17px;">

================================================================================

</div>

<div style="font-size: 17px;">


=== Technical Translation in Spanish ===

</div>

<div style="font-size: 17px;">

Estatus: 500 Error interno del servidor del servidor
Respuesta: {
    'error': 'Falló la conexión a la base de datos',
    'code': 'DB_001',
    'timestamp': '2024-03-20T10:30:00Z'
}

Nota técnica: Este error ocurre cuando la aplicación no puede conectarse a la base de datos.

</div>

<div style="font-size: 17px;">

================================================================================

</div>

In [35]:
from langchain.messages import SystemMessage, HumanMessage

# Create the system message with examples
system_message = SystemMessage(content="""You are a technical translator. You must maintain the exact same format and structure in your translations.
Only translate the explanatory text, keeping all technical terms, numbers, and formatting unchanged.

Example input and output pairs:

Input: "Error 404: Page not found"
Output: "Error 404: Página no encontrada"

Input: "Status: 200 OK
Response: {
    'data': 'success',
    'message': 'Operation completed'
}"
Output: "Status: 200 OK
Response: {
    'data': 'success',
    'message': 'Operación completada'
}"
""")

# Loop through each language
for language in languages:
    print_text(f"\n=== Technical Translation in {language} ===")

    # Create the human message using f-string
    human_message = HumanMessage(content=f"""Translate this technical information to {language}:

    Status: 500 Internal Server Error
    Response: {{
        'error': 'Database connection failed',
        'code': 'DB_001',
        'timestamp': '2024-03-20T10:30:00Z'
    }}

    Technical Note: This error occurs when the application cannot connect to the database.
    """)

    # Create messages list
    messages = [system_message, human_message]

    # Get response
    res = chat_llm.invoke(messages)
    time.sleep(3)

    print_text(res.content)
    print_text("=" * 80)  # Separator for readability

<div style="font-size: 17px;">


=== Technical Translation in English ===

</div>

<div style="font-size: 17px;">

    Status: 500 Internal Server Error
    Response: {
        'error': 'Database connection failed',
        'code': 'DB_001',
        'timestamp': '2024-03-20T10:30:00Z'
    }

    Technical Note: This error occurs when the application fails to establish a connection to the database.

</div>

<div style="font-size: 17px;">

================================================================================

</div>

<div style="font-size: 17px;">


=== Technical Translation in Esperanto ===

</div>

<div style="font-size: 17px;">

Statuso: 500 Interna Servo-ero de la Servilo
Ripozo: {
    'eraro': 'Falsita konekto al la datumbazo',
    'kodo': 'DB_001',
    'tempo': '2024-03-20T10:30:00Z'
}

Teknika Notacio: Ĉi tiu eraro okazas kiam la aplikilo ne sukcesas konektiĝi al la datumbazo.

</div>

<div style="font-size: 17px;">

================================================================================

</div>

<div style="font-size: 17px;">


=== Technical Translation in Spanish ===

</div>

<div style="font-size: 17px;">

    Status: 500 Error interno del servidor
    Response: {
        'error': 'Falló la conexión a la base de datos',
        'code': 'DB_001',
        'timestamp': '2024-03-20T10:30:00Z'
    }

    Nota técnica: Este error ocurre cuando la aplicación no puede conectarse a la base de datos.

</div>

<div style="font-size: 17px;">

================================================================================

</div>

In [36]:
# Create few-shot examples for technical content formatting
system_template = SystemMessagePromptTemplate.from_template(
    """You are a technical translator. You must maintain the exact same format and structure in your translations.
    Only translate the explanatory text, keeping all technical terms, numbers, and formatting unchanged.

    Example input and output pairs:

    Input: "Error 404: Page not found"
    Output: "Error 404: Página no encontrada"

    Input: "Status: 200 OK
    Response: {% raw %}{{
        'data': 'success',
        'message': 'Operation completed'
    }}{% endraw %}"
    Output: "Status: 200 OK
    Response: {% raw %}{{
        'data': 'success',
        'message': 'Operación completada'
    }}{% endraw %}"
    """,
    template_format="jinja2"
)

# Example of a technical input using Jinja2's control structures and filters
human_template = HumanMessagePromptTemplate.from_template(
    """Translate this technical information to {{ language|upper }}:

    Status: 500 Internal Server Error
    Response: {% raw %}{{
        'error': 'Database connection failed',
        'code': 'DB_001',
        'timestamp': '2024-03-20T10:30:00Z'
    }}{% endraw %}

    Technical Note: This error occurs when the application cannot connect to the database.

    {% if language == 'spanish' %}
    Note: Please use formal Spanish for technical documentation.
    {% elif language == 'french' %}
    Note: Please use formal French for technical documentation.
    {% else %}
    Note: Please maintain a formal tone in the translation.
    {% endif %}

    {% for term in technical_terms %}
    Keep the term "{{ term }}" unchanged in the translation.
    {% endfor %}
    """,
    template_format="jinja2"
)

# Create the chain using LCEL pipe syntax
chain = (
    ChatPromptTemplate.from_messages([system_template, human_template])
    | chat_llm
    | StrOutputParser()
)

# Loop through each language
for language in languages:
    print_text(f"\n=== Technical Translation in {language} ===")

    # Invoke the chain with our inputs
    result = chain.invoke({
        "language": language,
        "technical_terms": ['DB_001', 'Internal Server Error', 'Database connection']
    })
    time.sleep(3)

    print_text(result)
    print_text("=" * 80)  # Separator for readability

<div style="font-size: 17px;">


=== Technical Translation in English ===

</div>

<div style="font-size: 17px;">

Status: 500 Internal Server Error
Response: {{
    'error': 'Database connection failed',
    'code': 'DB_001',
    'timestamp': '2024-03-20T10:30:00Z'
}}

Technical Note: This error occurs when the application cannot establish a connection to the database.

</div>

<div style="font-size: 17px;">

================================================================================

</div>

<div style="font-size: 17px;">


=== Technical Translation in Esperanto ===

</div>

<div style="font-size: 17px;">

Status: 500 Interna Servera Eraro
Response: {{
    'eraro': 'Fremetiga konekto al la datumaro malsukcesis',
    'kodo': 'DB_001',
    'tempo': '2024-03-20T10:30:00Z'
}}

Teknologia Noto: Ĉi tiu eraro okazas, kiam la aplikaĵo ne sukcesas konektiĝi al la datumaro.

    
    Noto: Bonvolu konservi formalan tonon en la traduko.

</div>

<div style="font-size: 17px;">

================================================================================

</div>

<div style="font-size: 17px;">


=== Technical Translation in Spanish ===

</div>

<div style="font-size: 17px;">

    Estado: 500 Error interno del servidor
    Respuesta: {{
        'error': 'Falló la conexión con la base de datos',
        'code': 'DB_001',
        'timestamp': '2024-03-20T10:30:00Z'
    }}

    Nota técnica: Este error se produce cuando la aplicación no es capaz de conectarse con la base de datos.

    
    Observación: Por favor, mantenga un tono formal en la traducción.

    
    Se mantiene el término "DB_001" sin cambios en la traducción.
    
    Se mantiene el término "Internal Server Error" sin cambios en la traducción.
    
    Se mantiene el término "Database connection" sin cambios en la traducción.

</div>

<div style="font-size: 17px;">

================================================================================

</div>

In [37]:
# Get the formatted prompt for Spanish
print_text("\n=== Formatted Prompt for Spanish ===")

# Format the prompts with our inputs
formatted_prompt = ChatPromptTemplate.from_messages([
    system_template,
    human_template
]).format_prompt(
    language='spanish',
    technical_terms=['DB_001', 'Internal Server Error', 'Database connection']
)

# Print the formatted messages
for message in formatted_prompt.to_messages():
    print_text(f"\n{message.type.upper()} MESSAGE:")
    print_text("-" * 40)
    print_text(message.content)
    print_text("=" * 80)

<div style="font-size: 17px;">


=== Formatted Prompt for Spanish ===

</div>

<div style="font-size: 17px;">


SYSTEM MESSAGE:

</div>

<div style="font-size: 17px;">

----------------------------------------

</div>

<div style="font-size: 17px;">

You are a technical translator. You must maintain the exact same format and structure in your translations.
    Only translate the explanatory text, keeping all technical terms, numbers, and formatting unchanged.

    Example input and output pairs:

    Input: "Error 404: Page not found"
    Output: "Error 404: Página no encontrada"

    Input: "Status: 200 OK
    Response: {{
        'data': 'success',
        'message': 'Operation completed'
    }}"
    Output: "Status: 200 OK
    Response: {{
        'data': 'success',
        'message': 'Operación completada'
    }}"
    

</div>

<div style="font-size: 17px;">

================================================================================

</div>

<div style="font-size: 17px;">


HUMAN MESSAGE:

</div>

<div style="font-size: 17px;">

----------------------------------------

</div>

<div style="font-size: 17px;">

Translate this technical information to SPANISH:

    Status: 500 Internal Server Error
    Response: {{
        'error': 'Database connection failed',
        'code': 'DB_001',
        'timestamp': '2024-03-20T10:30:00Z'
    }}

    Technical Note: This error occurs when the application cannot connect to the database.

    
    Note: Please use formal Spanish for technical documentation.
    

    
    Keep the term "DB_001" unchanged in the translation.
    
    Keep the term "Internal Server Error" unchanged in the translation.
    
    Keep the term "Database connection" unchanged in the translation.
    
    

</div>

<div style="font-size: 17px;">

================================================================================

</div>

# Lanchain Chains

## Simple Chain example

In [38]:
# Create a function to handle calculations
def calculate(expression: str) -> str:
    """Calculate using numexpr, with support for basic math operations."""
    try:
        result = float(numexpr.evaluate(expression))
        return f"The result is: {result}"
    except Exception as e:
        return f"Error in calculation: {str(e)}"

# Create the prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful math assistant. When given a math problem, respond ONLY with the mathematical expression that would solve it. For example, if asked 'What is 2 raised to the 3rd power?', respond only with '2**3'."),
    ("user", "{question}")
])

# Wrap our calculation function with RunnableLambda for explicit LCEL pattern
calculate_runnable = RunnableLambda(calculate)

# Create the chain using LCEL with explicit RunnableLambda
math_chain = (
    prompt
    | chat_llm  # LLM to generate the expression
    | StrOutputParser()  # Convert to string
    | calculate_runnable  # Our calculation function wrapped in RunnableLambda
)

# Use the chain with our example
response = math_chain.invoke({
    "question": "What is 13 raised to the .3432 power?"
})
time.sleep(3)
print_text(response)

<div style="font-size: 17px;">

The result is: 2.4116004626599237

</div>

In [39]:
# Simple prompt without guidance
prompt = ChatPromptTemplate.from_messages([
    ("user", "{question}")
])

basic_chain = (
    prompt
    | chat_llm  # LLM tries to calculate directly
    | StrOutputParser()
)

response = basic_chain.invoke({
    "question": "What is 13 raised to the .3432 power?"
})
time.sleep(3)
print_text(response)  # The LLM tries to calculate it directly and might get it wrong!

<div style="font-size: 17px;">

To calculate 13 raised to the 0.3432 power, we need to use the concept of exponentiation. 

Exponentiation can be expressed as:

a^b = a multiplied by itself b times

However, in this case, we can also use the mathematical function exp(b * ln(a)), where ln(a) is the natural logarithm of a.

Let's calculate the value:

a = 13
b = 0.3432

First, we need to find the natural logarithm of 13:
ln(13) ≈ 2.5649

Now, we multiply b by ln(a):
b * ln(a) = 0.3432 * 2.5649 ≈ 0.8805

Now, we apply the exponential function:
exp(0.8805) ≈ 2.403

So, 13 raised to the 0.3432 power is approximately 2.403.

</div>

## Building Complex Chains with LCEL

In [40]:
def clean_text(text: str) -> str:
    # replace multiple new lines and multiple spaces with a single one
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)
    return text

In [41]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a creative writing assistant."),
    ("user", """Please paraphrase this text in the style of {style}: {text}""")
])

In [42]:
# Create the chain using LCEL
style_chain = (
    {
        "text": lambda x: clean_text(x["text"]),  # Extract and clean the text from input dict
        "style": lambda x: x["style"]  # Extract style from input dict
    }
    | prompt  # Format with our template
    | chat_llm  # Generate creative paraphrase
    | StrOutputParser()  # Convert to string
)

# Our input text with messy spacing
input_text = """
Chains allow us to combine multiple


components together to create a single, coherent application.

For example, we can create a chain that takes user input,       format it with a PromptTemplate,

and then passes the formatted response to an LLM. We can build more complex chains by combining     multiple chains together, or by


combining chains with other components.
"""

# Run the chain
response = style_chain.invoke({
    "text": input_text,
    "style": "a 90s rapper"
})
time.sleep(3)
print_text(response)

<div style="font-size: 17px;">

Yo, listen up, I got a tale to tell,
'Bout chains, the tech that makes the magic swell,
See, we got these components, multiple and strong,
But solo, they're weak, they ain't where it belongs.

But then we hit 'em with a chain, a link in the game,
Now we got multiple components, all in the same frame,
User input, formatted right, with a PromptTemplate tight,
 Pass it to an LLM, day and night, it's gettin' tight.

We build the chain, we add some flair,
More complex chains, we show we care,
Combine 'em together, make 'em strong and new,
Or mix 'em with other components, the possibilities are true.

So when you're workin' with tech, and you need it to shine,
Just remember chains, they'll make your app divine.

</div>

## Using RunnableParallel and RunnablePassthrough

In [43]:
# Create two different analysis prompts
sentiment_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a sentiment analysis expert. Analyze the emotional tone."),
    ("user", "What's the sentiment of: {text}")
])

summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a summarization expert."),
    ("user", "Summarize in one sentence: {text}")
])

# Use RunnableParallel to run both analyses simultaneously
analysis_chain = RunnableParallel(
    {
        "sentiment": sentiment_prompt | chat_llm | StrOutputParser(),
        "summary": summary_prompt | chat_llm | StrOutputParser(),
        "original": RunnablePassthrough()  # Pass through the original input
    }
)

# Test it
sample_text = {"text": "The product exceeded my expectations. Great quality!"}
results = analysis_chain.invoke(sample_text)
time.sleep(3)

print("Sentiment:", results["sentiment"])
print("Summary:", results["summary"])
print("Original:", results["original"]["text"])

Sentiment: The sentiment of the given statement is overwhelmingly **POSITIVE**.

The words and phrases used convey a sense of satisfaction and delight:

- "exceeded my expectations" implies that the product met or surpassed the user's high standards, which is a positive outcome.
- "Great quality" is a subjective evaluation, but it's a common phrase used to express admiration and approval.

The overall tone of the statement is enthusiastic and optimistic, indicating that the user is extremely pleased with the product.
Summary: The product met or exceeded expectations due to its high quality.
Original: The product exceeded my expectations. Great quality!


## Batch Processing with LCEL

In [44]:
# Create a simple question-answering chain
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer concisely."),
    ("user", "{question}")
])

qa_chain = qa_prompt | chat_llm | StrOutputParser()

# Batch of questions
questions = [
    {"question": "What is the capital of France?"},
    {"question": "Who wrote Romeo and Juliet?"},
    {"question": "What is the speed of light?"}
]

# Process all questions in batch
answers = qa_chain.batch(questions)
time.sleep(3)

# Display results
for q, a in zip(questions, answers):
    print_text(f"Q: {q['question']}")
    print_text(f"A: {a}\n")

<div style="font-size: 17px;">

Q: What is the capital of France?

</div>

<div style="font-size: 17px;">

A: The capital of France is Paris.


</div>

<div style="font-size: 17px;">

Q: Who wrote Romeo and Juliet?

</div>

<div style="font-size: 17px;">

A: Romeo and Juliet was written by William Shakespeare.


</div>

<div style="font-size: 17px;">

Q: What is the speed of light?

</div>

<div style="font-size: 17px;">

A: The speed of light in a vacuum is approximately 299,792 kilometers per second (km/s) or about 186,282 miles per second (mi/s).


</div>